# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [40]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd_2025-10-27_17-33-10.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

2025-10-29 09:51:34,655 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [41]:
schema_prompt = f"""
You are a senior Database Administrator (DBA). Act only as the DBA and use the Product Requirements Document (PRD) below as the single source of truth. Do not invent features or behaviors that are not implied by the PRD.

PRD CONTEXT:
{prd_content}

Task:
Design a normalized, production-ready SQL schema for an onboarding tool based on the PRD above.

Requirements / Constraints:
- Output must be SQLite-compatible SQL only.
- Return raw SQL statements only (no explanatory text, no Markdown fences).
- You must include at least the following tables: users and onboarding_tasks. Add supporting tables (e.g., departments, roles, task_templates, user_tasks, attachments) as needed to achieve at least third-normal-form (3NF).
- Use appropriate SQLite types (INTEGER, TEXT, REAL). Use TEXT for ISO-8601 timestamp columns and/or TIMESTAMP where applicable.
- For primary keys use INTEGER PRIMARY KEY or INTEGER PRIMARY KEY AUTOINCREMENT as appropriate.
- Define FOREIGN KEYs, UNIQUE constraints, NOT NULL where appropriate, and sensible DEFAULT values.
- Include created_at and updated_at timestamp columns with DEFAULT (CURRENT_TIMESTAMP) where appropriate.
- Provide indexes on foreign keys and columns expected to be searched/joined (e.g., email, status, department_id).
- Include PRAGMA foreign_keys = ON; and optionally DROP TABLE IF EXISTS statements preceding CREATE TABLE statements for idempotence.
- Use only SQL compatible with SQLite (no vendor-specific types or ON UPDATE clauses unsupported by SQLite).
- Do not include any narrative, comments outside SQL, or any other output. Inline SQL comments are acceptable if strictly necessary.
- Produce only raw CREATE TABLE (and optional DROP/CREATE INDEX/PRAGMA) statements that implement the schema.

Produce the final schema as raw SQLite SQL statements only.
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/schema.sql', overwrite=True) # rewrote on rerun
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---
PRAGMA foreign_keys = ON;

DROP TABLE IF EXISTS submission_feedback;
DROP TABLE IF EXISTS project_submissions;
DROP TABLE IF EXISTS form_submissions;
DROP TABLE IF EXISTS user_tasks;
DROP TABLE IF EXISTS role_task_assignments;
DROP TABLE IF EXISTS task_templates;
DROP TABLE IF EXISTS resources;
DROP TABLE IF EXISTS resource_categories;
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS roles;
DROP TABLE IF EXISTS departments;

CREATE TABLE departments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    created_at TEXT NOT NULL DEFAULT (strftime('%Y-%m-%dT%H:%M:%fZ', 'now')),
    updated_at TEXT NOT NULL DEFAULT (strftime('%Y-%m-%dT%H:%M:%fZ', 'now'))
);

CREATE TABLE roles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    description TEXT,
    created_at TEXT NOT NULL DEFAULT (strftime('%Y-%m-%dT%H:%M:%fZ', 'now')),
    updated_at TEXT NOT NULL DEFAULT (strftime('%Y-%m-%dT%H:%M:%fZ', 'now'))
);

CREATE 

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [42]:
seed_data_prompt = f"""
You are a senior Database Administrator generating realistic seed data for a newly created SQLite onboarding tool database.

PRD CONTEXT:
{prd_content}

SCHEMA:
{cleaned_schema}

Task:
Generate 5-10 total INSERT statements that populate the schema with realistic onboarding data.

Requirements / Constraints:
- Output ONLY raw SQL INSERT statements (no CREATE/DROP, no comments, no explanatory text, no markdown fences).
- Use only tables present in the schema.
- Provide meaningful onboarding-focused data: departments, roles, users (new hire, manager, HR, IT), task templates, onboarding task instances / user task mappings, attachments (if that table exists).
- Satisfy foreign key order: insert parent tables first (e.g., departments -> roles -> users -> task_templates -> onboarding_tasks / user_tasks -> attachments).
- Use explicit integer primary key values starting at 1 (stable, no AUTOINCREMENT reliance in output).
- Multi-row INSERTs encouraged to stay within the 5–10 statement limit.
- Use realistic corporate-style emails and ISO-8601 timestamps where explicit values are needed. If a column has a DEFAULT CURRENT_TIMESTAMP you may omit it.
- Use only the following statuses: 'pending','in_progress','completed' if a status column exists.
- Do not exceed 10 INSERT statements total.
- No UPDATE/DELETE statements.
- Do not invent tables or columns not in the provided schema.

Return ONLY the INSERT statements. Output must be valid SQLite SQL and adhere to the schema provided above.
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql', overwrite=True) # rewrote on rerun
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
INSERT INTO departments (id, name) VALUES (1, 'Engineering'), (2, 'Human Resources'), (3, 'Marketing');
INSERT INTO roles (id, name, description) VALUES (1, 'Software Engineer', 'Builds and maintains software applications.'), (2, 'Engineering Team Lead', 'Manages a team of software engineers.'), (3, 'HR Onboarding Specialist', 'Manages the new hire onboarding process.');
INSERT INTO users (id, full_name, email, sso_user_id, user_type, role_id, department_id, manager_id, mentor_id, start_date, experience_level, is_active) VALUES (1, 'Emily Chen', 'emily.chen@corp.com', 'echen', 'manager', 2, 1, NULL, NULL, '2022-08-15', 'Senior', 1), (2, 'David Rodriguez', 'david.rodriguez@corp.com', 'drodriguez', 'hr_specialist', 3, 2, NULL, NULL, '2021-05-20', 'Mid-level', 1), (3, 'Liam Smith', 'liam.smith@corp.com', 'lsmith', 'employee', 1, 1, 1, NULL, '2023-01-10', 'Senior', 1), (4, 'Sarah Jones', 'sarah.jones@corp.com', 'sjones', 'new_hire', 1, 1, 1, 3, '2025-10-27', 'J

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [ ]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Enable foreign key constraints
        cursor.execute("PRAGMA foreign_keys = ON;")
        print(f"Successfully connected to database at {db_path}")

        # Read the content of the schema file using load_artifact.
        schema_sql = load_artifact(schema_path)
        if not schema_sql:
            print(f"Error: Could not read schema file at {schema_path}")
            return
        
        # Execute the schema SQL script.
        # Use cursor.executescript() for multi-statement SQL strings.
        cursor.executescript(schema_sql)
        print("Tables created successfully.")

        # Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            seed_sql = load_artifact(seed_path)
            if seed_sql:
                cursor.executescript(seed_sql)
                print("Seed data inserted into database successfully.")
            else:
                print(f"Warning: Could not read seed data file at {seed_path}")
        else:
            print(f"Warning: Seed data file not found at {seed_path}")

        # Commit the changes to the database.
        conn.commit()
        print("Database changes committed successfully.")
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
            print("Database changes rolled back due to error.")
    except Exception as e:
        print(f"Unexpected error: {e}")
        if conn:
            conn.rollback()
            print("Database changes rolled back due to unexpected error.")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Remove existing database file to start fresh
if os.path.exists(db_file):
    os.remove(db_file)
    print(f"Removed existing database file: {db_file}")

# Execute the function
create_database(db_file, schema_file, seed_file)

Removed existing database file: c:\Users\647020\OneDrive - BOOZ ALLEN HAMILTON\Desktop\Workspace\AG-AISOFTDEV-OCT\artifacts\onboarding.db
Successfully connected to database at c:\Users\647020\OneDrive - BOOZ ALLEN HAMILTON\Desktop\Workspace\AG-AISOFTDEV-OCT\artifacts\onboarding.db
Tables created successfully.
Seed data inserted into database successfully.
Database changes committed successfully.
Database connection closed.


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.